In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import utility
import math
from time import time
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from sklearn.metrics import classification_report

In [ ]:
def load_column():
    df = pd.read_csv('./datasets/column_2C.dat', sep=" ", names=['pelvic_incidence', 'pelvic_tilt', 'lumbar_lordosis_angle', 'sacral_slope', 'pelvic_radius', 'degree_spondylolisthesis','target'])
    df['target']=np.where(df['target']=='AB',1,0)
    return df

def load_blood_transfusion():
    df = pd.read_csv('./datasets/blood_transfusion.csv')
    return df

def load_ionosphere():
    df = pd.read_csv('./datasets/Ionosphere.csv')
    return df

def load_parkinson():
    df = pd.read_csv('./datasets/parkinsons.csv')
    return df

def load_pima():
    df = pd.read_csv('./datasets/diabetes.csv')
    df.rename(columns={'Outcome': 'target'}, inplace=True)
    return df

def load_glass():
    df = pd.read_csv('./datasets/glass.csv')
    # Avoid to_categorical error due to missing class '4'
    unique_labels = sorted(df['target'].unique())  # Ensure sorted order
    label_map = {original: new for new, original in enumerate(unique_labels)}
    
    df['target'] = df['target'].map(label_map)
    return df

def load_climate():
    df = pd.read_csv('./datasets/climate_model_simulation_crashes.csv')
    return df

def load_modeling():
    df = pd.read_csv('./datasets/User_Knowledge_Modeling.csv')
    unique_labels = sorted(df['target'].unique())  # Ensure sorted order
    label_map = {original: new for new, original in enumerate(unique_labels)}
    
    df['target'] = df['target'].map(label_map)
    return df

def load_banknote():
    df = pd.read_csv('./datasets/banknote_authentication.csv')
    return df

def load_sonar():
    df = pd.read_csv('./datasets/sonar.csv')
    return df

In [ ]:
#SklearnDatasets
df = load_sonar()
dir_path = 'Sonar'

In [ ]:
df

In [ ]:
scaler = MinMaxScaler()
scaler.fit(df.values[:, :-1])
scaled_df = scaler.transform(df.values[:, :-1])
lower_bound = scaled_df.min()
upper_bound = scaled_df.max()
print(lower_bound, upper_bound)

In [ ]:
scaled_df.shape

In [ ]:
df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
#targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
#df_scaled['target'] = targets
targets = df['target'].values
df_scaled['target'] = df['target']
num_classes = len(df_scaled['target'].unique())
display(df_scaled)

In [ ]:
np.random.seed(50)
X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
n_neurons = X_train.shape[1]
n_hidden_layers = 1
print(f'n_hidden_layers: {n_hidden_layers}, n_neurons {n_neurons}')
model = tf.keras.Sequential([
    tf.keras.Input(shape=(X_train.shape[1],)),
    layers.Dense(n_neurons, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model_path = f'new_models/{dir_path}.keras'
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=40)
ck = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_accuracy', save_best_only=True)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)
start = time()
model.fit(X_train,y_train, epochs=400, batch_size=32,validation_data=(X_test, y_test), verbose=1, callbacks=[ck, es])
print(f'Training time: {time()-start}')
model = tf.keras.models.load_model(model_path)
print('Training Results')
model.evaluate(X_train, y_train, verbose=2)
print('Test Results')
model.evaluate(X_test, y_test, verbose=2)
model.save(f'new_models/{dir_path}.h5', save_format='h5')

In [ ]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  

y_test_classes = np.argmax(y_test, axis=1)

report = classification_report(y_test_classes, y_pred_classes,digits=4)
print(report)

# Save the report to a text file
with open(f'new_models/{dir_path}_report.txt', 'w') as f:
    f.write(report)